In [7]:
'''
from whereever my mouse is... goes to (100,100) and right clicks. 
then it continuously prints the mouse position until the script is stopped 
'''

import pyautogui
import time

# Failsafe: moving the mouse to a corner will raise an exception
pyautogui.FAILSAFE = True

# Move the mouse
pyautogui.moveTo(100, 100)

# Click
pyautogui.click()

# Move and click
pyautogui.moveTo(200, 200, duration=2)  # Move over 2 seconds
pyautogui.click()

# Right-click
pyautogui.rightClick()

# Get position
print(pyautogui.position())

# PyAutoGUI doesn't have built-in event listening, but you can poll for position
try:
    while True:
        x, y = pyautogui.position()
        print(f"Current mouse position: {x}, {y}")
        time.sleep(1)  # Check every second
except KeyboardInterrupt:
    print("\nDone.")


Point(x=200, y=200)
Current mouse position: 200, 200
Current mouse position: 200, 200
Current mouse position: 200, 200
Current mouse position: 200, 200
Current mouse position: 200, 200
Current mouse position: 200, 200
Current mouse position: 200, 200
Current mouse position: 200, 200
Current mouse position: 573, 190
Current mouse position: 611, 187
Current mouse position: 611, 187
Current mouse position: 611, 187
Current mouse position: 611, 187
Current mouse position: 611, 187
Current mouse position: 86, 139
Current mouse position: 106, 970
Current mouse position: 106, 970
Current mouse position: 142, 673
Current mouse position: 145, 343
Current mouse position: 291, 388
Current mouse position: 291, 388
Current mouse position: 291, 388
Current mouse position: 291, 388
Current mouse position: 641, 493
Current mouse position: 648, 499
Current mouse position: 648, 499
Current mouse position: 299, 100
Current mouse position: 460, 510
Current mouse position: 460, 510
Current mouse position: 

In [ ]:
'''
in trombone champ, players control pitch by moving the mouse up and down

all movement is vertical along a fixed x, along [y_bottom, y_top]
'''

In [12]:
import pyaudio
import numpy as np
import librosa
import time

# Audio settings
CHUNK = 1024
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 44100

# Calibration time in seconds
CALIBRATION_TIME = 10

def calibrate_otamatone():
    p = pyaudio.PyAudio()
    
    try:
        stream = p.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        frames_per_buffer=CHUNK)

        print(f"Please play your Otamatone for {CALIBRATION_TIME} seconds...")
        print("Try to cover the entire range from lowest to highest notes.")
        print("Calibration starting in:")
        for i in range(3, 0, -1):
            print(f"{i}...")
            time.sleep(1)
        print("Go!")

        min_pitch = float('inf')
        max_pitch = 0
        start_time = time.time()
        end_time = start_time + CALIBRATION_TIME

        while time.time() < end_time:
            try:
                data = np.frombuffer(stream.read(CHUNK, exception_on_overflow=False), dtype=np.float32)
                pitches, magnitudes = librosa.piptrack(y=data, sr=RATE)
                index = magnitudes[:, 0].argmax()
                pitch = pitches[index, 0]

                if pitch > 0:  # Ignore zero pitch (no sound detected)
                    min_pitch = min(min_pitch, pitch)
                    max_pitch = max(max_pitch, pitch)
                
                # Print a dot every second to show progress
                print(".", end="", flush=True)
                time.sleep(0.1)  # Small delay to prevent too many dots
            
            except IOError as e:
                print(f"Warning: {e}")
                continue

    except Exception as e:
        print(f"An error occurred: {e}")
    
    finally:
        try:
            stream.stop_stream()
            stream.close()
        except:
            pass
        p.terminate()

    print("\nCalibration complete!")
    if min_pitch != float('inf') and max_pitch != 0:
        print(f"Detected pitch range:")
        print(f"MIN_PITCH = {min_pitch:.2f} Hz")
        print(f"MAX_PITCH = {max_pitch:.2f} Hz")
    else:
        print("No valid pitches detected. Please try again.")

    return min_pitch, max_pitch

# Run the calibration
min_pitch, max_pitch = calibrate_otamatone()

Please play your Otamatone for 10 seconds...
Try to cover the entire range from lowest to highest notes.
Calibration starting in:
3...
2...
1...
Go!
...............................................................................................
Calibration complete!
Detected pitch range:
MIN_PITCH = 144.09 Hz
MAX_PITCH = 1224.39 Hz
